# Model training lab

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

## Prerequisits

In [1]:
# Imports
import os
import warnings
import tools
import models as c_models
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Bidirectional

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Root CSV files directory
dirname = "./data/"

# Constant frame count.
frames = 100

/home/datagroup/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/datagroup/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/datagroup/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/datagroup/anaconda3/lib/python3.7/site-packages/tensorflow/python/f

## Preparation Stage
### Load data and normalize
For training it's required to extend/reduce every dataset to n frames, where n is `frames`.

In [2]:
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))


In [3]:
# Print the content of the first dataframe
data[0][1]

,face_x,face_y,landmark_x_1,landmark_y_1,landmark_x_2,landmark_y_2,landmark_x_3,landmark_y_3,landmark_x_4,landmark_y_4,...,landmark_x_38,landmark_y_38,landmark_x_39,landmark_y_39,landmark_x_40,landmark_y_40,landmark_x_41,landmark_y_41,landmark_x_42,landmark_y_42
0,0.429839,0.310724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.429000,0.311571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.426787,0.312287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.424972,0.309270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.424178,0.308381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Split data
Split the dataset up into the following segments:
1. Training Data: 60%
2. Validation Data: 20%
3. Test Data: 20%


In [4]:
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size=0.40, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.50, random_state=42)

In [5]:
# Display data distribution
print("Total:", len(labels))
print("Training:", len(y_train), len(y_train) / len(labels) * 100)
print("Val:", len(y_val), len(y_val) / len(labels) * 100)
print("Test:", len(y_test), len(y_test) / len(labels) * 100)

Total: 347
Training: 208 59.94236311239193
Val: 69 19.88472622478386
Test: 70 20.172910662824208


### Tokenize (One Hot)

In [6]:
import tools
tokenizer = tools.tokenize(dirname)
print(tokenizer.word_index)
encoded_train=tokenizer.texts_to_sequences([y_train])[0]
encoded_val=tokenizer.texts_to_sequences([y_val])[0]
encoded_test=tokenizer.texts_to_sequences([y_test])[0]
y_train = to_categorical(encoded_train)
y_val = to_categorical(encoded_val)
y_test = to_categorical(encoded_test)
print(y_train)

{'computer': 1, 'hallo': 2, 'welt': 3, 'deutschland': 4}
[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]]


In [7]:
# Making numpy arrays
x_train=np.array(x_train)
y_train=np.array(y_train)
x_val=np.array(x_val)
y_val=np.array(y_val)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [13]:
print(x_train)

[[[0.548248 0.378688 0.121178 ... 0.       0.       0.      ]
  [0.547959 0.382478 0.112727 ... 0.       0.       0.      ]
  [0.549318 0.381478 0.115534 ... 0.       0.       0.      ]
  ...
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]]

 [[0.425378 0.32222  0.       ... 0.       0.       0.      ]
  [0.424511 0.322983 0.       ... 0.       0.       0.      ]
  [0.424099 0.321831 0.       ... 0.       0.       0.      ]
  ...
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]]

 [[0.539983 0.332844 0.725697 ... 0.817143 0.188124 0.832209]
  [0.539281 0.329601 0.235574 ... 0.       0.       0.      ]
  [0.532962 0.335245 0.701214 ... 0.       0.       0.      ]
  ...
  [0.       0.       0.       ... 0.       0.   

## Training Stage
Configure the model and train it.

Metrics:
![accuracy](assets/accuracy.png)
![precision and recall formula](assets/precision_recall_formula.png)
![precision and recall](assets/precision_recall.png)


### Configure model
Configures the model with the specified parameters.

#### LSTM

In [9]:
model = Sequential()
model.add(layers.LSTM(64, return_sequences=True,
               input_shape=(100, 86)))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32))  # return a single vector of dimension 32
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

Instructions for updating:
Colocations handled automatically by placer.


### or
#### Bidirectional LSTM

In [28]:
model = Sequential()
model.add(Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(100, 86)))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

### or
#### Simple RNN

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(200, 42)))
model.add(Dense(64, activation="softmax")) #softmax, linear 어떤걸 기준으로 하지
model.add(Dense(128, activation="linear")) #softmax, linear 어떤걸 기준으로 하지
model.add(Dense(21))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

### Train

In [10]:
history=model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_val,y_val))

Train on 208 samples, validate on 69 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
208/208 [==============================] - 2s 10ms/sample - loss: 1.5490 - acc: 0.2788 - precision: 1.0000 - recall: 0.0144 - val_loss: 1.3790 - val_acc: 0.4493 - val_precision: 1.0000 - val_recall: 0.0290
Epoch 2/100
208/208 [==============================] - 1s 4ms/sample - loss: 1.3559 - acc: 0.3798 - precision: 0.6857 - recall: 0.1154 - val_loss: 1.1436 - val_acc: 0.4638 - val_precision: 0.9375 - val_recall: 0.2174
Epoch 3/100
208/208 [==============================] - 1s 4ms/sample - loss: 1.1524 - acc: 0.4375 - precision: 0.9231 - recall: 0.1731 - val_loss: 1.1086 - val_acc: 0.4493 - val_precision: 0.8000 - val_recall: 0.2319
Epoch 4/100
208/208 [==============================] - 1s 4ms/sample - loss: 1.1056 - acc: 0.4856 - precision: 0.8261 - recall: 0.1827 - val_loss: 1.0667 - val_acc: 0.4638 - val_precision: 0.8000 - val_recall: 0.2319
Epoch 5/100
208/208 [=================

208/208 [==============================] - 1s 4ms/sample - loss: 0.5945 - acc: 0.8077 - precision: 0.8116 - recall: 0.8077 - val_loss: 0.6962 - val_acc: 0.7681 - val_precision: 0.7681 - val_recall: 0.7681
Epoch 38/100
208/208 [==============================] - 1s 5ms/sample - loss: 0.6118 - acc: 0.8077 - precision: 0.8077 - recall: 0.8077 - val_loss: 0.5178 - val_acc: 0.8116 - val_precision: 0.8235 - val_recall: 0.8116
Epoch 39/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.4581 - acc: 0.8606 - precision: 0.8647 - recall: 0.8606 - val_loss: 0.4225 - val_acc: 0.9130 - val_precision: 0.9118 - val_recall: 0.8986
Epoch 40/100
208/208 [==============================] - 1s 5ms/sample - loss: 0.4927 - acc: 0.8606 - precision: 0.8599 - recall: 0.8558 - val_loss: 0.4384 - val_acc: 0.8696 - val_precision: 0.8696 - val_recall: 0.8696
Epoch 41/100
208/208 [==============================] - 1s 5ms/sample - loss: 0.5526 - acc: 0.8173 - precision: 0.8213 - recall: 0.8173 - val

Epoch 75/100
208/208 [==============================] - 1s 5ms/sample - loss: 0.5548 - acc: 0.8125 - precision: 0.8155 - recall: 0.8077 - val_loss: 0.6160 - val_acc: 0.8261 - val_precision: 0.8235 - val_recall: 0.8116
Epoch 76/100
208/208 [==============================] - 1s 5ms/sample - loss: 0.4899 - acc: 0.8510 - precision: 0.8502 - recall: 0.8462 - val_loss: 0.4203 - val_acc: 0.8986 - val_precision: 0.8986 - val_recall: 0.8986
Epoch 77/100
208/208 [==============================] - 1s 5ms/sample - loss: 0.4353 - acc: 0.8606 - precision: 0.8606 - recall: 0.8606 - val_loss: 0.4679 - val_acc: 0.8696 - val_precision: 0.8696 - val_recall: 0.8696
Epoch 78/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.3835 - acc: 0.8894 - precision: 0.8878 - recall: 0.8750 - val_loss: 0.4320 - val_acc: 0.8986 - val_precision: 0.8986 - val_recall: 0.8986
Epoch 79/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.3611 - acc: 0.8990 - precision: 0.8990 - recall:

### Evaluate

In [12]:
model.evaluate(x=x_test, y=y_test, batch_size=32)

70/70 [==============================] - 0s 1ms/sample - loss: 0.6487 - acc: 0.7429 - precision: 0.7429 - recall: 0.7429


[0.6486993293677058, 0.74285716, 0.74285716, 0.74285716]

### Save model

In [11]:
model.save("sign_lang_recognition.h5")